
<div style="width: 100%; max-width: 100%; background-color: #fefefe; border: 1px solid #333; border-radius: 10px; padding: 20px; font-family: Arial, sans-serif; box-sizing: border-box;">
  <h3 style="color: #2c3e50; text-align: center;">Data Processing</h2>
  
  <p style="color: #34495e; line-height: 1.6;">
    In the midterm project, electricity and economic data were processed. In the final project, population data was incorporated, and data cleaning was performed again. The results were saved in “data_imputation_full.csv”.
    
  </p>
  
</div>

In [1]:
import pandas as pd
import requests
from io import StringIO
import numpy as np

# 1. Load data
url_imputation = 'data_imputation.csv'
data_imputation = pd.read_csv(url_imputation, index_col=[0, 1])

data_urls = {
    'world_population': 'https://github.com/xiaothua/dataset/raw/main/world_population.csv'
}

def load_data_from_url(url):
    response = requests.get(url)
    response.raise_for_status()  # Ensure the request was successful
    csv_data = StringIO(response.text)
    return pd.read_csv(csv_data)

data_population = load_data_from_url(data_urls['world_population'])

In [2]:
# Drop rows with missing values in relevant columns to prevent any unexpected issues
relevant_columns = ['Country/Territory', '2022 Population', '2020 Population', '2015 Population',
                    '2010 Population', '2000 Population', '1990 Population', '1980 Population',
                    'Area (km²)', 'World Population Percentage']
data_population_filtered = data_population.dropna(subset=['Country/Territory'])[relevant_columns]

country_name_corrections = {
    "Brunei": "Brunei Darussalam",
    'Turkey': "Türkiye",
    'DR Congo': "COD",
    'Cape Verde': "Cabo Verde",
    'United States Virgin Islands': "United States Minor Outlying Islands"
}

data_population_filtered['Country/Territory'] = data_population_filtered['Country/Territory'].replace(country_name_corrections)

# Add new indices for population-related information to data_imputation
new_indices = ['Population: Population', 
               'Population: Percentage', 
               'Population: Area (km²)',
               'Population: Growth Rate', 
               'Population: Density']

# Extend the existing MultiIndex to include new attributes for population data
existing_index = data_imputation.index
new_index = pd.MultiIndex.from_product([
    existing_index.levels[0], existing_index.levels[1].tolist() + new_indices
], names=existing_index.names)

data_imputation = data_imputation.reindex(new_index)

In [3]:
# Fill in the 'Population: Area (km²)' data from data_population_filtered
for country in data_population_filtered['Country/Territory'].unique():
    if country in data_imputation.index.levels[0]:
        area_value = data_population_filtered.loc[data_population_filtered['Country/Territory'] == country, 'Area (km²)'].values[0]
        data_imputation.loc[(country, 'Population: Area (km²)')] = area_value

# Fill in the 'Population: Population' data using linear interpolation
years = [1980, 1990, 2000, 2010, 2015, 2020, 2022]
for country in data_population_filtered['Country/Territory'].unique():
    if country in data_imputation.index.levels[0]:
        population_values = data_population_filtered.loc[data_population_filtered['Country/Territory'] == country, [f'{year} Population' for year in years]].values.flatten()
        interp_years = range(1992, 2022 + 1)  # Interpolating from 1992 to 2022
        interpolated_population = np.interp(interp_years, years, population_values)
        for year, population in zip(interp_years, interpolated_population):
            if str(year) in data_imputation.columns:
                data_imputation.loc[(country, 'Population: Population'), str(year)] = population

# Calculate total population for each year and compute percentage
for year in data_imputation.columns:
    if year.isdigit():
        total_population = data_imputation.loc[(slice(None), 'Population: Population'), year].sum()
        for country in data_population_filtered['Country/Territory'].unique():
            if country in data_imputation.index.levels[0]:
                country_population = data_imputation.loc[(country, 'Population: Population'), year]
                if total_population > 0:
                    data_imputation.loc[(country, 'Population: Percentage'), year] = (country_population / total_population) * 100

# Calculate population density for each country and year
for country in data_population_filtered['Country/Territory'].unique():
    if country in data_imputation.index.levels[0]:
        area = data_imputation.loc[(country, 'Population: Area (km²)'), :].iloc[0]  # Assuming area is constant across all years
        if pd.notna(area) and area > 0:
            for year in data_imputation.columns:
                if year.isdigit():
                    population = data_imputation.loc[(country, 'Population: Population'), year]
                    if pd.notna(population):
                        data_imputation.loc[(country, 'Population: Density'), year] = population / area

# Calculate population growth rate for each country and year
for country in data_population_filtered['Country/Territory'].unique():
    if country in data_imputation.index.levels[0]:
        for year in range(1993, 2023):  # From 1993 to 2022, to calculate growth rate compared to the previous year
            current_year = str(year)
            previous_year = str(year - 1)
            if current_year in data_imputation.columns and previous_year in data_imputation.columns:
                current_population = data_imputation.loc[(country, 'Population: Population'), current_year]
                previous_population = data_imputation.loc[(country, 'Population: Population'), previous_year]
                if pd.notna(current_population) and pd.notna(previous_population) and previous_population > 0:
                    growth_rate = ((current_population - previous_population) / previous_population) * 100
                    data_imputation.loc[(country, 'Population: Growth Rate'), current_year] = growth_rate
        # Set 1992 growth rate equal to 1993 value
        if '1992' in data_imputation.columns and '1993' in data_imputation.columns:
            data_imputation.loc[(country, 'Population: Growth Rate'), '1992'] = data_imputation.loc[(country, 'Population: Growth Rate'), '1993']

# Check for NaN values in data_imputation and print details if present
nan_values = data_imputation[data_imputation.isna().any(axis=1)]
if not nan_values.empty:
    print('Detailed NaN information:')
    print(nan_values)
else:
    print('No NaN values found in data_imputation.')

No NaN values found in data_imputation.


In [4]:
data_imputation.to_csv('data_imputation_full.csv')

In [5]:
data_imputation.head(40)

1992  \
Country                                                             
Albania Economics: GDP                               6.521750e+08   
        Economics: GDP Growth                       -7.187111e+00   
        Economics: GDP Per Capita                    2.008522e+02   
        Economics: GDP Per Capita Growth            -6.622551e+00   
        Economics: PPP                               5.917192e+09   
        Economics: PPP Growth                       -5.071988e+00   
        Economics: PPP Per Capita                    1.822335e+03   
        Economics: PPP Per Capita Growth            -4.494563e+00   
        Electricity: Distribution Losses             1.411000e+00   
        Electricity: Distribution Losses Growth      1.040689e+01   
        Electricity: Distribution Losses Per Capita  4.345498e-07   
        Electricity: Exports                         5.600000e-01   
        Electricity: Exports Growth                 -5.685670e+01   
        Electricity: Exports Per Capita              1.724648e-07   
        Electricity: Imports                         1.040000e-01   
        Electricity: Imports Growth                 -8.400000e+01   
        Electricity: Imports Per Capita              3.202918e-08   
        Electricity: Installed Capacity              1.892000e+00   
        Electricity: Installed Capacity Growth       0.000000e+00   
        Electricity: Installed Capacity Per Capita   5.826847e-07   
        Electricity: Net Consumption                 1.443000e+00   
        Electricity: Net Consumption Growth         -1.586006e+01   
        Electricity: Net Consumption Per Capita      4.444049e-07   
        Electricity: Net Generation                  3.310000e+00   
        Electricity: Net Generation Growth          -9.090909e+00   
        Electricity: Net Generation Per Capita       1.019390e-06   
        Electricity: Net Imports                    -4.560000e-01   
        Electricity: Net Imports Growth             -2.962963e+01   
        Electricity: Net Imports Per Capita         -1.404356e-07   
        Population: Population                       3.272457e+06   
        Population: Percentage                       6.158273e-02   
        Population: Area (km²)                       2.874800e+04   
        Population: Growth Rate                     -3.454438e-01   
        Population: Density                          1.138325e+02   
Algeria Economics: GDP                               4.800308e+10   
        Economics: GDP Growth                        1.800002e+00   
        Economics: GDP Per Capita                    1.776028e+03   
        Economics: GDP Per Capita Growth            -5.647888e-01   
        Economics: PPP                               1.893524e+11   
        Economics: PPP Growth                        4.119933e+00   

                                                             1993  \
Country                                                             
Albania Economics: GDP                               1.185315e+09   
        Economics: GDP Growth                        9.559412e+00   
        Economics: GDP Per Capita                    3.672792e+02   
        Economics: GDP Per Capita Growth             1.022995e+01   
        Economics: PPP                               6.636506e+09   
        Economics: PPP Growth                        1.215634e+01   
        Economics: PPP Per Capita                    2.056373e+03   
        Economics: PPP Per Capita Growth             1.284277e+01   
        Electricity: Distribution Losses             1.655000e+00   
        Electricity: Distribution Losses Growth      1.729270e+01   
        Electricity: Distribution Losses Per Capita  5.128146e-07   
        Electricity: Exports                         1.410000e-01   
        Electricity: Exports Growth                 -7.482143e+01   
        Electricity: Exports Per Capita              4.368995e-08   
        Electricity: Imports                         8.200

In [6]:
data_imputation.index.levels[1]

Index(['Economics: GDP', 'Economics: GDP Growth', 'Economics: GDP Per Capita',
       'Economics: GDP Per Capita Growth', 'Economics: PPP',
       'Economics: PPP Growth', 'Economics: PPP Per Capita',
       'Economics: PPP Per Capita Growth', 'Electricity: Distribution Losses',
       'Electricity: Distribution Losses Growth',
       'Electricity: Distribution Losses Per Capita', 'Electricity: Exports',
       'Electricity: Exports Growth', 'Electricity: Exports Per Capita',
       'Electricity: Imports', 'Electricity: Imports Growth',
       'Electricity: Imports Per Capita', 'Electricity: Installed Capacity',
       'Electricity: Installed Capacity Growth',
       'Electricity: Installed Capacity Per Capita',
       'Electricity: Net Consumption', 'Electricity: Net Consumption Growth',
       'Electricity: Net Consumption Per Capita',
       'Electricity: Net Generation', 'Electricity: Net Generation Growth',
       'Electricity: Net Generation Per Capita', 'Electricity: Net Imports